In [1]:
import pandas as pd
import numpy as np
oof_train = np.loadtxt('data/model_save/train_bert_prob_0.8977973628929045.txt') # 0.bert_0.8977973628929045.csv
oof_test = np.loadtxt('data/model_save/test_bert_prob_0.8977973628929045.txt') #  bert_0.0.0.7308399121801145

In [113]:
# train['情感倾向'] = np.argmax(oof_train, axis=1) - 1
# train.to_csv('data/train_presto.csv', index=)

In [2]:
train = pd.read_csv('data/nCoV_100k_train.labled.csv',header=0)
# train = pd.read_csv('data/new_train.csv')
test = pd.read_csv('data/nCov_10k_test.csv',header=0)
train = train[(train['情感倾向'].isin(['-1','0','1']))]
train = train[~train.index.isin(['35202','41624','33936'])].reset_index(drop=True)
labels = train['情感倾向'].astype('int').values

In [5]:
train_y = labels+1
optR = OptimizedRounder()
optR.fit(oof_train, train_y)
coefficients = optR.coefficients()
print(coefficients)

[1.06643358 0.93466748 1.06549164]


In [6]:
oof_train2 = coefficients * oof_train
cv_score = f1_score(train_y, np.argmax(oof_train, axis=-1), average='macro')
cv_score

0.7225638338130592

In [7]:
# f1_score(train_y, np.argmax(oof_train2, axis=-1), average='macro')
cv_score = optR.predict(oof_train, train_y)
cv_score

0.7233132140763497

In [8]:
# coefficients = [1.11111411, 0.936682753, 0.96274868]
oof_test2= oof_test * coefficients
test['y'] = np.argmax(oof_test2, axis=1) - 1
test['id'] = test['微博id'].copy()#map(lambda x: str(x)+' ')
test['y'].value_counts(1)

 0    0.5868
 1    0.2452
-1    0.1680
Name: y, dtype: float64

In [9]:
test2 = pd.read_csv('data/submit/bert_0.7328015743130485.csv')
test['y2'] = test2['y']
test[test['y'] == test['y2']].shape[0]/test.shape[0]
test['y2'].value_counts(1)

 0    0.5759
 1    0.2401
-1    0.1840
Name: y2, dtype: float64

In [10]:
test[test['y'] == test['y2']].shape[0]/test.shape[0]

0.9294

In [133]:
np.savetxt('./data/model_save/train_bert_prob_{}.txt'.format(cv_score), oof_train2)
np.savetxt('./data/model_save/test_bert_prob_{}.txt'.format(cv_score), oof_test2)

In [11]:
test[['id', 'y']].to_csv('./data/submit/bert_{}.csv'.format(cv_score), index=False)

In [135]:
cv_score

0.7233132140763497

In [4]:
# -*- coding: utf-8 -*-
"""
@Time ： 2020/3/15 22:59
@Auth ： joleo
@File ：optimized_rounder.py
"""
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from functools import partial
import scipy as sp

class OptimizedRounder(object):
    """
    An optimizer for rounding thresholds
    to maximize Quadratic Weighted Kappa (QWK) score
    # https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved
    # https://www.kaggle.com/teejmahal20/regression-with-optimized-rounder
    """

    def __init__(self):
        self.coef_ = 0

    def _macro_f1_score(y_true, y_pred, n_labels):
        # https://www.kaggle.com/corochann/fast-macro-f1-computation
        total_f1 = 0.
        for i in range(n_labels):
            yt = y_true == i
            yp = y_pred == i

            tp = np.sum(yt & yp)

            tpfp = np.sum(yp)
            tpfn = np.sum(yt)
            if tpfp == 0:
                print('[WARNING] F-score is ill-defined and being set to 0.0 in labels with no predicted samples.')
                precision = 0.
            else:
                precision = tp / tpfp
            if tpfn == 0:
                print(f'[ERROR] label not found in y_true...')
                recall = 0.
            else:
                recall = tp / tpfn
            if precision == 0. or recall == 0.:
                f1 = 0.
            else:
                f1 = 2 * precision * recall / (precision + recall)
            total_f1 += f1
        return total_f1 / n_labels

    def _f1_loss(self, coef, X, y):
        """
        Get loss according to
        using current coefficients

        :param coef: A list of coefficients that will be used for rounding
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        X_p = coef * np.copy(X)

        return -f1_score(y, np.argmax(X_p, axis=-1), average='macro')

    def fit(self, X, y):
        """
        Optimize rounding thresholds

        :param X: The raw predictions
        :param y: The ground truth labels
        """
        loss_partial = partial(self._f1_loss, X=X, y=y)
        initial_coef = [1.0 for _ in range(len(set(y)))]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, y):
        """
        Make predictions with specified thresholds

        :param X: The raw predictions
        :param coef: A list of coefficients that will be used for rounding
        """
        X_p = self.coef_['x'] * np.copy(X)

        return f1_score(y, np.argmax(X_p, axis=-1), average='macro')

    def coefficients(self):
        """
        Return the optimized coefficients
        """
        return self.coef_['x']